In [2]:
import numpy as np
from scipy.special import digamma, polygamma
import time
from decimal import Decimal

**Load and Preprocess the data**

In [3]:
f = open('ap.txt', 'r')
text = f.readlines()
f.close()

In [4]:
D1 = set([x for x in text if '<DOC>\n' not in x])
D2 = set([x for x in text if '</DOC' not in x])
D3 = set([x for x in text if 'TEXT>' not in x])
D = list(set.intersection(D1, D2, D3))

In [5]:
import toolz as tz
import toolz.curried as c
import string
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import re
import pandas as pd
import numpy as np

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
stops = [
    "a", "about", "above", "across", "after", "afterwards", "again", "against",
    "all", "almost", "alone", "along", "already", "also", "although", "always",
    "am", "among", "amongst", "amoungst", "amount", "an", "and", "another",
    "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are",
    "around", "as", "at", "back", "be", "became", "because", "become",
    "becomes", "becoming", "been", "before", "beforehand", "behind", "being",
    "below", "beside", "besides", "between", "beyond", "bill", "both",
    "bottom", "but", "by", "call", "can", "cannot", "cant", "co", "con",
    "could", "couldnt", "cry", "de", "describe", "detail", "do", "done",
    "down", "due", "during", "each", "eg", "eight", "either", "eleven", "else",
    "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone",
    "everything", "everywhere", "except", "few", "fifteen", "fifty", "fill",
    "find", "fire", "first", "five", "for", "former", "formerly", "forty",
    "found", "four", "from", "front", "full", "further", "get", "give", "go",
    "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter",
    "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his",
    "how", "however", "hundred", "i", "ie", "if", "in", "inc", "indeed",
    "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter",
    "latterly", "least", "less", "ltd", "made", "many", "may", "me",
    "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly",
    "move", "much", "must", "my", "myself", "name", "namely", "neither",
    "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone",
    "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on",
    "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our",
    "ours", "ourselves", "out", "over", "own", "part", "per", "perhaps",
    "please", "put", "rather", "re", "same", "see", "seem", "seemed",
    "seeming", "seems", "serious", "several", "she", "should", "show", "side",
    "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone",
    "something", "sometime", "sometimes", "somewhere", "still", "such",
    "system", "take", "ten", "than", "that", "the", "their", "them",
    "themselves", "then", "thence", "there", "thereafter", "thereby",
    "therefore", "therein", "thereupon", "these", "they", "thick", "thin",
    "third", "this", "those", "though", "three", "through", "throughout",
    "thru", "thus", "to", "together", "too", "top", "toward", "towards",
    "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us",
    "very", "via", "was", "we", "well", "were", "what", "whatever", "when",
    "whence", "whenever", "where", "whereafter", "whereas", "whereby",
    "wherein", "whereupon", "wherever", "whether", "which", "while", "whither",
    "who", "whoever", "whole", "whom", "whose", "why", "will", "with",
    "within", "without", "would", "yet", "you", "your", "yours", "yourself",
    "yourselves"]

In [7]:
d = tz.pipe(
    D,
    c.map(lambda x: x.strip()),
    c.map(lambda x: x.lower()),
    c.map(lambda x: x.translate(str.maketrans('', '', string.punctuation))),
    c.map(lambda x: re.sub('[0-9]+', '', x)),
    c.map(lambda x: x.split()),
    c.map(lambda x: [word for word in x if word not in stops]),
    list
)

In [8]:
d_sub = d[:500]

In [9]:
tf = {id: tz.frequencies(doc) for id, doc in enumerate(d_sub)}
df = pd.DataFrame(tf).fillna(0)
words = df.index

In [10]:
ds = df.values.T
ds = ds.astype(int)

In [11]:
def DataTrans(x):
    """Turn the data into the desired structure"""
    
    N_d = np.count_nonzero(x)
    V = len(x)
    
    row = 0
    
    doc = np.zeros((N_d, V))
    for i in range(V):
        if x[i] == 0:
            pass
        else:
            doc[row, i] = x[i]
            row += 1
    
    return doc

In [12]:
docs = list(map(DataTrans, ds))

**Use the package**

In [ ]:
from VIonLDA import *

In [15]:
a, B = M_step_Realdata(docs=docs,k=10,tol=1e-3,tol_estep=1e-3,max_iter=100,initial_alpha_shape=100,initial_alpha_scale=0.01)

In [44]:
import heapq

In [45]:
def find_index(x):
    """find the index of the largest 10 values in a list"""
    
    x = x.tolist()
    max_values = heapq.nlargest(50, x)
    index = [0] * 50
    for i in range(50):
        index[i] = x.index(max_values[i])
    
    return index

In [46]:
rep_words_index = list(map(find_index, B))

In [47]:
words[rep_words_index[0]]

Index(['said', 'percent', 'million', 'year', 'new', 'market', 'stock',
       'prices', 'billion', 'rose', 'orders', 'economy', 'index', 'rate',
       'bank', 's', 'oil', 'business', 'increase', 'tuesday', 'rates',
       'economic', 'time', 'month', 'shares', 'analysts', 'record', 'average',
       'american', 'york', 'exchange', 'government', 'share', 'trade',
       'company', 'north', 'high', 'sales', 'stocks', 'report', 'reported',
       'soviet', 'cents', 'earlier', 'week', 'months', 'monday', 'expected',
       'decline', 'deficit'],
      dtype='object')

In [48]:
words[rep_words_index[1]]

Index(['said', 'percent', 'billion', 'law', 'year', 'united', 'states', 'drug',
       'million', 'court', 'years', 'offer', 'federal', 'women', 'rights',
       'committee', 'told', 'house', 'prison', 'say', 'people', 'debt',
       'health', 'american', 'new', 'state', 'public', 'administration',
       'report', 'workers', 'government', 'case', 'foreign', 'congress',
       'measure', 'legislation', 'decision', 'president', 'attorney',
       'abortion', 'stock', 'office', 'senate', 'department', 'legal', 'think',
       'issue', 'program', 'time', 'supreme'],
      dtype='object')

In [49]:
words[rep_words_index[2]]

Index(['said', 'president', 'united', 'government', 'program', 'agreement',
       'soviet', 'aid', 'south', 'american', 'foreign', 'states', 'house',
       'million', 'officials', 'talks', 'negotiations', 'monday', 'minister',
       'economic', 'year', 'summit', 'trade', 'official', 'senate', 'military',
       'told', 'billion', 'new', 'administration', 'secretary', 'sen',
       'people', 'time', 'plan', 'week', 'congress', 'end', 'iraq', 'release',
       'agency', 'iran', 'navy', 'peace', 'statement', 'eastern', 'thats',
       'years', 'conference', 'plant'],
      dtype='object')

In [50]:
words[rep_words_index[3]]

Index(['said', 'west', 'dollar', 'new', 'late', 'german', 'east', 'fair',
       'soviet', 'bid', 'years', 'officials', 'bank', 'today', 'yen', 'london',
       'million', 'trust', 'gold', 'government', 'tuesday', 'editor',
       'thursday', 'cloudy', 'germany', 'federal', 'york', 'ap', 'party',
       'snow', 'monday', 'north', 'friday', 'people', 'price', 'year',
       'claims', 'fell', 'major', 'spokesman', 'later', 'home', 'news',
       'manville', 'ounce', 'percent', 'compared', 'central', 'ms', 'board'],
      dtype='object')

In [51]:
words[rep_words_index[4]]

Index(['said', 'dukakis', 'new', 'bush', 'president', 'campaign', 'state',
       'democratic', 'national', 'republican', 'presidential', 'convention',
       'york', 'years', 'senate', 'vice', 'states', 'democrats', 'year',
       'people', 'week', 'going', 'home', 'think', 'george', 'support', 'sen',
       'candidate', 'nomination', 'governor', 'party', 'bentsen',
       'administration', 'force', 'chairman', 'court', 'race', 'primary',
       'vote', 'work', 'conservative', 'way', 'million', 'report', 'reagan',
       'running', 'tuesday', 'michael', 'dont', 'cbs'],
      dtype='object')

In [52]:
words[rep_words_index[5]]

Index(['said', 'police', 'people', 'hospital', 'children', 'officials', 'area',
       'man', 'years', 'friday', 'family', 'new', 'state', 'killed', 'yearold',
       'night', 'authorities', 'did', 'parents', 'sunday', 'city', 'service',
       'time', 'miles', 'told', 'office', 'saturday', 'victims', 'news',
       'reported', 'center', 'building', 'county', 'school', 'like',
       'president', 'arrested', 'say', 'million', 'year', 'today', 'asked',
       'death', 'ms', 'hotel', 'thought', 'homeless', 'angeles', 'church',
       'way'],
      dtype='object')

In [53]:
words[rep_words_index[6]]

Index(['said', 'new', 'years', 'state', 'people', 'sand', 'says', 'east',
       'rating', 'hunt', 'time', 'west', 'breast', 'dont', 'country', 'just',
       'cancer', 'art', 'fish', 'hubbert', 'political', 'red', 'president',
       'summer', 'play', 'group', 'like', 'north', 'problems', 'second',
       'days', 'use', 'heat', 'including', 'device', 'chemical', 'year',
       'good', 'japanese', 'area', 'percent', 'agency', 'day', 'campaign',
       'world', 'economic', 'french', 'want', 'ago', 'early'],
      dtype='object')

In [54]:
words[rep_words_index[7]]

Index(['said', 'people', 'government', 'million', 'years', 'president', 'year',
       'company', 'soviet', 'barry', 'black', 'milken', 'gorbachev',
       'thursday', 'state', 'dont', 'south', 'new', 'corp', 'make', 'chairman',
       'court', 'city', 'group', 'think', 'independence', 'billion', 'united',
       'agreement', 'news', 'africa', 'federal', 'told', 'drexel', 'agreed',
       'monday', 'time', 'friday', 'union', 'officials', 'spokesman',
       'macmillan', 'chief', 'wednesday', 'power', 'board', 'white', 'trial',
       'police', 'like'],
      dtype='object')

In [55]:
words[rep_words_index[8]]

Index(['said', 'bush', 'new', 'people', 'president', 'air', 'million', 'state',
       'government', 'political', 'systems', 'federal', 'water', 'going',
       'national', 'nations', 'way', 'energy', 'states', 'billion', 'called',
       'presidential', 'administration', 'years', 'city', 'percent', 'public',
       'foreign', 'year', 'congress', 'general', 'american', 'officials',
       'black', 'time', 'help', 'community', 'election', 'venus', 'director',
       'court', 'council', 'judge', 'program', 'rights', 'security',
       'campaign', 'group', 'told', 'country'],
      dtype='object')

In [56]:
words[rep_words_index[9]]

Index(['said', 'government', 'police', 'people', 'court', 'case', 'military',
       'trial', 'president', 'year', 'day', 'party', 'charges', 'national',
       'death', 'years', 'new', 'today', 'officials', 'told', 'defense',
       'friday', 'john', 'killed', 'attorney', 'documents', 'army',
       'gorbachev', 'conference', 'soldiers', 'demjanjuk', 'judge', 'time',
       'leaders', 'soviet', 'convicted', 'million', 'moscow', 'committee',
       'charged', 'force', 'yearold', 'spokesman', 'federal', 'thursday',
       'security', 'monday', 'accused', 'men', 'minister'],
      dtype='object')